In [7]:
import cv2 as cv
import numpy as np

In [133]:
# enter path of original image
org_img = "test\\jfif\\the_rock.jfif"

# enter path of cover image
cov_img = "test\\jfif\\the_rock.jfif"

from os import path

if not path.isfile(org_img):
    print(f"File {org_img} does not exist")
if not path.isfile(cov_img):
    print(f"File {cov_img} does not exist")

In [140]:
# check for similarities between images by looking at each bit plane and calculating the relative difference

def img2bitplane(img):
    # Get image dimensions
    height, width, channels = img.shape

    # Splitting the color channels
    a = None
    if channels > 3:
        # png image with fourth alpha channel
        b, g , r, a = cv.split(img)
    else:
        # jpeg or gif
        b, g, r = cv.split(img)

    
    # assume bitdepth of 8
    bitdepth = 8

    # converting integer decimal values into 8 bit binary values
    b_bits = np.unpackbits(np.array([b], dtype=np.uint8), axis = 1)
    g_bits = np.unpackbits(np.array([g], dtype=np.uint8), axis = 1)
    r_bits = np.unpackbits(np.array([r], dtype=np.uint8), axis = 1)
    if a != None:
        a_bits = np.unpackbits(np.array([a], dtype=np.uint8), axis = 1)
    # print(b_bits.shape)

    # lists to store sliced bit planes
    # b_out = []
    # g_out = []
    # r_out = []
    # if a.any():
    #     a_out = []
    # # loop over all 8 bits
    # for p in range(bitdepth):
    #     # create binary mask with 1 at position 'p' / (2**p) means 2 to the power of p
    #     plane = np.full((height, width), 2 ** p, np.uint8)
    #     # Bitwise and operation to only obtain bits at position 'p'
    #     b_slice = cv.bitwise_and(plane,b)
    #     g_slice = cv.bitwise_and(plane,g)
    #     r_slice = cv.bitwise_and(plane,r)
    #     if a.any():
    #         a_slice = cv.bitwise_and(plane,a)
        
    #     # save the sliced planes in a list
    #     b_out.append(b_slice)
    #     g_out.append(g_slice)
    #     r_out.append(r_slice)
    #     if a.any():
    #         a_out.append(a_slice)
    
    # if a.any():
    #     return [b_out, g_out, r_out, a_out]
    # else:
    #     return [b_out, g_out, r_out]
    if a != None:
        return b_bits, g_bits, r_bits, a_bits
    else:
        return b_bits, g_bits, r_bits

def load_as_bitplane(org_img):
    ending = org_img.split(".")[-1]
    if ending == "png":
        # loads alpha channel as well
        img = cv.imread(org_img, cv.IMREAD_UNCHANGED)
    else:
        img = cv.imread(org_img)

    print(f"size: {img.shape[:2]}\nchannels: {img.shape[-1]}")
    bit_channels = img2bitplane(img)
    shape = img.shape
    return bit_channels, shape

In [143]:
def compare_bitplanes():
    bit_channels_org, shape = load_as_bitplane(org_img)
    bit_channels_cov, _ = load_as_bitplane(cov_img)

    bit_count_org = np.product(shape)*8  # bit_depth
    for ch_ix, channel in enumerate(bit_channels_org):
        print("xxx:", len(channel))
        pn_counter = [0,0,0,0,0,0,0,0]
        for pn_ix, plane in enumerate(channel):
            print("xx:", len(plane))
            for row_ix, row in enumerate(plane):
                print("x: ", len(row))
                for col_ix, col in enumerate(row):
                    print("np.uint8:", col)
                    if col==bit_channels_cov[ch_ix][pn_ix][row_ix][col_ix]:
                        pn_counter[col_ix] += 1
        print(f"channel: {ch_ix}\t plane: ", 100*pn_counter/(bit_count_org/len(bit_channels_org)))

    



In [144]:
compare_bitplanes()

size: (450, 800)
channels: 3
size: (450, 800)
channels: 3
xxx: 1
xx: 3600
x:  800
np.uint8: 1
np.uint8: 1
np.uint8: 1
np.uint8: 1
np.uint8: 1
np.uint8: 1
np.uint8: 1
np.uint8: 1
np.uint8: 1


IndexError: list index out of range